In [1]:
from clr import AddReference
AddReference("System")
AddReference("QuantConnect.Common")
AddReference("QuantConnect.Jupyter")
AddReference("QuantConnect.Indicators")
from System import *
from QuantConnect import *
from QuantConnect.Data.Market import TradeBar, QuoteBar
from QuantConnect.Jupyter import *
from QuantConnect.Indicators import *
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
qb = QuantBook()

In [2]:
volatility_etfs = ["SQQQ", "TQQQ", "TVIX", "VIXY", "SPLV",
                   "UVXY", "EEMV", "EFAV", "USMV","SPY"]
# Add Equity ------------------------------------------------
for i in range(len(volatility_etfs)):
    qb.AddEquity(volatility_etfs[i],Resolution.Daily)
history = qb.History(qb.Securities.Keys, 252*15,Resolution.Daily)
history_SPY = history.loc['SPY']

In [3]:
rsv = (history_SPY['close']-history_SPY['close'].rolling(9,min_periods = 1).min()) /((history_SPY['close'].rolling(9,min_periods = 1).max()-history_SPY['close'].rolling(9,min_periods = 1).min()))
rsv = rsv.fillna(value = 0)
rsv = rsv*100

K = pd.DataFrame(np.zeros((len(rsv.index),1)),index = rsv.index)
K.iloc[0] = rsv.iloc[0]+50
# K = K.shift()*(2/3)+rsv*(1/3)

D = pd.DataFrame(np.zeros((len(rsv.index),1)),index = rsv.index)
D.iloc[0]=50

for i in range(len(K.index)-1):
    K.iloc[i+1] = K.iloc[i]*(2/3)+rsv.iloc[i+1]*(1/3)


for i in range(len(K.index)-1):
    D.iloc[i+1] = D.iloc[i]*(2/3)+K.iloc[i+1]*(1/3)

#建立SPY KD值表格
spy_kd = pd.concat([history_SPY['close'],rsv,K,D],axis = 1)
spy_kd.columns = ['close','rsv','k','D']

In [4]:
def signal():
    buy_sig = (spy_kd['k']>spy_kd['D']) & (spy_kd['k'].shift(1)<spy_kd['D'].shift(1))
    sell_sig = (spy_kd['k']<spy_kd['D']) & (spy_kd['k'].shift(1)>spy_kd['D'].shift(1))
    total_sig = spy_kd['k'].copy()
    for i in total_sig.index:
        if buy_sig[i] == True:
            total_sig[i]= 'Buy'
        elif sell_sig[i] == True:
            total_sig[i]= 'Sell'
        else:
            total_sig[i]= ' '
    return total_sig
signal = signal()

In [5]:
def profit_ratio(start_capital,stock_bull,stock_bear):
#先設定起始資金 start_capital
#目前手上有的股票種類
    hold_stock = []
#目前手上有的資金
    hold_capital = start_capital
#總資金 = 手上資金+股票總市值
    total_capital = start_capital
#資金變化表
    if (stock_bull!='') & (stock_bear!=''):
        min_index = min(len(history.loc[stock_bull].index),len(history.loc[stock_bear].index))
    elif (stock_bull!=''):
        min_index = len(history.loc[stock_bull].index)
    else:
        min_index = len(history.loc[stock_bear].index)
    total_list = pd.DataFrame(np.zeros((min_index,5)),columns=[stock_bull + ' close',stock_bear + ' close','hold capital','total capital','buy or sell'])
    total_list.index = history_SPY.iloc[-min_index:].index
    for i in total_list.index:
        if stock_bull!='':
            bull_price = history.loc[stock_bull].loc[i,'close']
        if stock_bear!='':
            bear_price = history.loc[stock_bear].loc[i,'close']
        if signal[i]=='Buy':
            if stock_bull!='':
                hold_stock.append(stock_bull)
                hold_capital-=bull_price
            total_list.loc[i,'buy or sell'] = 'buy'
            if stock_bear != '':
                hold_capital+=(len([x for x in hold_stock if x==stock_bear]))*bear_price
                hold_stock = [x for x in hold_stock if x!=stock_bear]
        elif signal[i]=='Sell':
            if stock_bear!='':
                hold_stock.append(stock_bear)
                hold_capital-=bear_price
            total_list.loc[i,'buy or sell'] = 'Sell'
            if stock_bull != '':
                hold_capital+=(len([x for x in hold_stock if x!=stock_bear]))*bull_price
                hold_stock = [x for x in hold_stock if x!=stock_bull]
        else:
            total_list.loc[i,'buy or sell'] = ''
        total_capital = hold_capital
        for g in hold_stock:
            total_capital+=history.loc[g].loc[i,'close']
        if stock_bull!='':
            total_list.loc[i,stock_bull + ' close']=bull_price
        if stock_bear!='':
            total_list.loc[i,stock_bear + ' close']=bear_price
        total_list.loc[i,'hold capital']=hold_capital
        total_list.loc[i,'total capital']=total_capital
    return total_list

In [8]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
hist = history.loc['SPY']['close'].loc['2014-01-01':'2018-12-31']
regr.fit(np.array([ [x] for x in range(len(hist.index))]),hist.values.reshape(-1,1))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [43]:
from datetime import datetime
index_count = 0
index_list = []
for index,time in enumerate(history.loc['SPY']['close'].loc['2014-01-01':'2019-12-31'].index):
    if time > datetime.strptime('2018-12-31', '%Y-%m-%d'):
        index_count = index
        break
index_list = history.loc['SPY']['close'].loc['2014-01-01':'2019-12-31'].index.copy()

In [44]:
index_list[index_count:]

DatetimeIndex(['2019-01-01', '2019-01-03', '2019-01-04', '2019-01-05',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-12', '2019-01-15',
               ...
               '2019-10-08', '2019-10-09', '2019-10-10', '2019-10-11',
               '2019-10-12', '2019-10-15', '2019-10-16', '2019-10-17',
               '2019-10-18', '2019-10-19'],
              dtype='datetime64[ns]', name='time', length=203, freq=None)

In [45]:
predict_value = regr.predict([[x+index_count] for x in range(len(index_list[index_count:]))])

In [47]:
compare_sheet = pd.concat((history.loc['SPY']['close'].loc['2019-01-01':'2019-12-31']
           ,pd.DataFrame(predict_value,index = index_list[index_count:])),axis = 1)
compare_sheet.columns = ['Act close','Predict close']

In [48]:
compare_sheet

,Act close,Predict close
time,,
2019-01-01,246.679099,269.858714
2019-01-03,246.817173,269.950513
2019-01-04,240.771504,270.042313
2019-01-05,248.996770,270.134112
2019-01-08,250.781870,270.225912
2019-01-09,253.158716,270.317712
2019-01-10,254.332345,270.409511
2019-01-11,255.397487,270.501311
2019-01-12,255.348175,270.593111
